## Kaggle


In [1]:
!pip install kaggle
from google.colab import files
files.upload()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"jeonhyotaek","key":"d2ddaf4c586e0bf63051e1a4c6a74dd4"}'}

In [2]:
ls -1ha kaggle.json

kaggle.json


In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [4]:
!kaggle competitions download -c 6th-goorm-project-2-korean-mrc

 33% 5.00M/15.3M [00:00<00:00, 28.1MB/s]
100% 15.3M/15.3M [00:00<00:00, 69.4MB/s]


In [5]:
!unzip 6th-goorm-project-2-korean-mrc.zip

Archive:  6th-goorm-project-2-korean-mrc.zip
  inflating: baseline.csv            
  inflating: blank.csv               
  inflating: test.json               
  inflating: train.json              


## data

In [6]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 70.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 83.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 16.4 MB/s eta 0:00:00


## gogo

In [70]:
tokenizer_name = "monologg/koelectra-small-v2-distilled-korquad-384"
model_name = "monologg/koelectra-small-v2-distilled-korquad-384"

In [71]:
import json
import random

import torch
import numpy as np
from tqdm.notebook import tqdm
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, AdamW

In [72]:
with open("train.json", 'rb') as f:
    input_dict = json.load(f)
input_dict["data"][0]

{'title': '제주도 장마 시작 … 중부는 이달 말부터',
 'paragraphs': [{'context': '올여름 장마가 17일 제주도에서 시작됐다. 서울 등 중부지방은 예년보다 사나흘 정도 늦은 이달 말께 장마가 시작될 전망이다.17일 기상청에 따르면 제주도 남쪽 먼바다에 있는 장마전선의 영향으로 이날 제주도 산간 및 내륙지역에 호우주의보가 내려지면서 곳곳에 100㎜에 육박하는 많은 비가 내렸다. 제주의 장마는 평년보다 2~3일, 지난해보다는 하루 일찍 시작됐다. 장마는 고온다습한 북태평양 기단과 한랭 습윤한 오호츠크해 기단이 만나 형성되는 장마전선에서 내리는 비를 뜻한다.장마전선은 18일 제주도 먼 남쪽 해상으로 내려갔다가 20일께 다시 북상해 전남 남해안까지 영향을 줄 것으로 보인다. 이에 따라 20~21일 남부지방에도 예년보다 사흘 정도 장마가 일찍 찾아올 전망이다. 그러나 장마전선을 밀어올리는 북태평양 고기압 세력이 약해 서울 등 중부지방은 평년보다 사나흘가량 늦은 이달 말부터 장마가 시작될 것이라는 게 기상청의 설명이다. 장마전선은 이후 한 달가량 한반도 중남부를 오르내리며 곳곳에 비를 뿌릴 전망이다. 최근 30년간 평균치에 따르면 중부지방의 장마 시작일은 6월24~25일이었으며 장마기간은 32일, 강수일수는 17.2일이었다.기상청은 올해 장마기간의 평균 강수량이 350~400㎜로 평년과 비슷하거나 적을 것으로 내다봤다. 브라질 월드컵 한국과 러시아의 경기가 열리는 18일 오전 서울은 대체로 구름이 많이 끼지만 비는 오지 않을 것으로 예상돼 거리 응원에는 지장이 없을 전망이다.',
   'qas': [{'question': '북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?',
     'answers': [{'text': '한 달가량', 'answer_start': 478},
      {'text': '한 달', 'answer_start': 478}],
     'guid': '798db07f0b9046759deed9d4a

In [73]:
input_dict["data"][1]

{'title': '부산정보산업진흥원, 과기부 지역SW서비스사업화 지원사업 4개 과제 선정',
 'paragraphs': [{'context': '부산시와 (재)부산정보산업진흥원(원장 이인숙)이 ‘2020~2021년 지역SW서비스사업화 지원사업’ 공모사업에 4개 과제가 선정되어 본격적인 사업 착수에 나선다. 과학기술정보통신부가 주관하는 ‘지역SW서비스사업화 지원사업’은 강소SW기업 및 초기 스타트업의 SW서비스 사업화 지원과 신시장 진출 지원을 통해 기업 경쟁력 강화와 지역경제 활성화를 도모하는 사업이다. 올해부터 2개년으로 진행되며, 국비와 시비, 민자 등 2년간 약 37억원의 예산이 투입된다. 앞서 진흥원은 부산의 미래 먹거리산업인 스마트해양, 지능형기계, 지능정보서비스 분야로 사전 수요조사를 진행했고, 평가를 통해 선정된 5개 과제를 공모사업에 신청했다. 그 결과 부산의 4개 과제가 최종 선정되는 쾌거를 거뒀다. 당 사업은 전국 진흥기관을 대상으로 공모를 시작해, 총 17개 지역에서 42개 과제가 선정되었으며, 4개 과제가 선정된 곳은 부산과 강원지역 뿐이다. 금번 선정된 과제들은 ‘인공지능융합센서와 서보 이송 로봇을 이용한 전단보강재의 자동용접시스템 개발’ 등 총 4개 과제다. 부산시가 지원하고, 부산정보산업진흥원과 지역기업, 대학, 연구소 등이 컨소시엄을 구성하여 기술개발 및 사업화 지원을 추진한다. 2개의 Track으로 구분되는 이번사업은 Track 1(SW중소기업)에서 ㈜에이아이플랫폼, 엔컴(주), Track 2(스타트업)에서는 ㈜토즈, 삼보테크놀로지를 지원한다. ○ ‘Track 1‘의 (주)에이아이플랫폼이 주관기업으로 진행하는 <인공지능 기반 망막 내 아밀로이드 플라크 영상 분석을 통한 치매조기진단 플랫폼 상용화>는 치매 확진의 원인이 되는 중요 단백질(아밀로이드 플라크)을 자체개발 관측장비로 진단한다. 이를 통해 치매를 조기 발견하여, 각종 경제적 비용과 치료 및 예방 등 사회적 문제를 해 결하고 시민들이 쉽게 접근 가능한 실효성 있는 치매관리

In [74]:
from copy import deepcopy
def split_input_dict(input_dict, ratio = 0.1, seed = 42):
    split_point = int(len(input_dict['data']) * ratio)
    random.seed(seed)
    random.shuffle(input_dict['data'])
    valid_dict = deepcopy(input_dict)
    train_dict = input_dict

    valid_dict['data'] = input_dict['data'][:split_point]
    train_dict['data'] = input_dict['data'][split_point:]
    return train_dict, valid_dict

In [102]:
def read_input(path):
    with open(path, 'rb') as f:
        input_dict = json.load(f)
    train_dict,valid_dict =split_input_dict(input_dict)
    train_contexts = []
    train_questions = []
    train_answers = []
    for group in tqdm(train_dict['data']):       #딕셔너리 하나씩 꺼낸다.
        for passage in group['paragraphs']:     #딕셔너리의 paragraphs
            context = passage['context']        #paragraphs의 context
            for qa in passage['qas']:           #paragraphs의 qas
                question = qa['question']       #paragraphs의 question
                for answer in qa['answers']:    #question의 answers
                    ### context 넘겨서 자르기 ###
                    target_context = context
                    if answer['answer_start'] > 1400:
                        target_context = target_context[1200:]
                        answer['answer_start'] -= 1200 
                    elif answer['answer_start'] > 1200:
                        target_context = target_context[1000:]
                        answer['answer_start'] -= 1000
                    elif answer['answer_start'] > 1000:
                        target_context = target_context[800:]
                        answer['answer_start'] -= 800
                    elif answer['answer_start'] > 800:
                        target_context = target_context[600:]
                        answer['answer_start'] -= 600
                    
                    
                    train_contexts.append(target_context)    #answers의 한 answer당 해당하는 context 저장
                    ### context 넘겨서 자르기 ###
                    train_questions.append(question)  #answers의 한 answer당 해당하는 question 저장
                    train_answers.append(answer)      #answers의 한 answer 저장
  
    valid_contexts = []
    valid_questions = []
    valid_answers = []
    for group in tqdm(valid_dict['data']):       #딕셔너리 하나씩 꺼낸다.
        for passage in group['paragraphs']:     #딕셔너리의 paragraphs
            context = passage['context']        #paragraphs의 context
            for qa in passage['qas']:           #paragraphs의 qas
                question = qa['question']       #paragraphs의 question
                for answer in qa['answers']:    #question의 answers
                    ### context 넘겨서 자르기 ###
                    target_context = context
                    if answer['answer_start'] > 1400:
                        target_context = target_context[1200:]
                        answer['answer_start'] -= 1200 
                    elif answer['answer_start'] > 1200:
                        target_context = target_context[1000:]
                        answer['answer_start'] -= 1000
                    elif answer['answer_start'] > 1000:
                        target_context = target_context[800:]
                        answer['answer_start'] -= 800
                    elif answer['answer_start'] > 800:
                        target_context = target_context[600:]
                        answer['answer_start'] -= 600

                    
                    valid_contexts.append(target_context)    #answers의 한 answer당 해당하는 context 저장
                    ### context 넘겨서 자르기 ###
                    valid_questions.append(question)  #answers의 한 answer당 해당하는 question 저장
                    valid_answers.append(answer)      #answers의 한 answer 저장

    return train_contexts, train_questions, train_answers, valid_contexts, valid_questions, valid_answers

In [103]:
def add_end_idx(answers, contexts):
    for answer, context in zip(answers, contexts):
        gold_text = answer['text']
        start_idx = answer['answer_start']
        end_idx = start_idx + len(gold_text)

        if context[start_idx:end_idx] == gold_text:
            answer['answer_end'] = end_idx
        elif context[start_idx-1:end_idx-1] == gold_text:
            print("there is an unitended error in dataset") #이렇게까지 할 필요가 있나?
            answer['answer_start'] = start_idx - 1
            answer['answer_end'] = end_idx - 1
        elif context[start_idx-2:end_idx-2] == gold_text:
            print("there is an unitended error in dataset")
            answer['answer_start'] = start_idx - 2
            answer['answer_end'] = end_idx - 2

In [104]:
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)

In [105]:
question = "북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?"
context = "올여름 장마가 17일 제주도에서 시작됐다. 서울 등 중부지방은 예년보다 사나흘 정도 늦은 이달 말께 장마가 시작될 전망이다.17일 기상청에 따르면 제주도 남쪽 먼바다에 있는 장마전선의 영향으로 이날 제주도 산간 및 내륙지역에 호우주의보가 내려지면서 곳곳에 100㎜에 육박하는 많은 비가 내렸다. 제주의 장마는 평년보다 2~3일, 지난해보다는 하루 일찍 시작됐다. 장마는 고온다습한 북태평양 기단과 한랭 습윤한 오호츠크해 기단이 만나 형성되는 장마전선에서 내리는 비를 뜻한다.장마전선은 18일 제주도 먼 남쪽 해상으로 내려갔다가 20일께 다시 북상해 전남 남해안까지 영향을 줄 것으로 보인다. 이에 따라 20~21일 남부지방에도 예년보다 사흘 정도 장마가 일찍 찾아올 전망이다. 그러나 장마전선을 밀어올리는 북태평양 고기압 세력이 약해 서울 등 중부지방은 평년보다 사나흘가량 늦은 이달 말부터 장마가 시작될 것이라는 게 기상청의 설명이다. 장마전선은 이후 한 달가량 한반도 중남부를 오르내리며 곳곳에 비를 뿌릴 전망이다. 최근 30년간 평균치에 따르면 중부지방의 장마 시작일은 6월24~25일이었으며 장마기간은 32일, 강수일수는 17.2일이었다.기상청은 올해 장마기간의 평균 강수량이 350~400㎜로 평년과 비슷하거나 적을 것으로 내다봤다. 브라질 월드컵 한국과 러시아의 경기가 열리는 18일 오전 서울은 대체로 구름이 많이 끼지만 비는 오지 않을 것으로 예상돼 거리 응원에는 지장이 없을 전망이다."
tokenizer(context, question)

{'input_ids': [2, 156, 18876, 8381, 29956, 550, 29982, 2757, 29951, 29962, 392, 30201, 29948, 5, 188, 55, 4473, 4660, 29961, 8674, 29997, 29948, 23326, 30672, 455, 1821, 29961, 6, 30124, 103, 30277, 8381, 29956, 392, 30292, 627, 29947, 29948, 5, 550, 29982, 5337, 29951, 541, 30013, 2757, 4613, 23905, 29951, 24, 29950, 8381, 9972, 29953, 814, 9563, 17196, 2757, 12364, 298, 7418, 16239, 29951, 10954, 21036, 29956, 1358, 29954, 13956, 3196, 29951, 1, 7532, 29959, 29950, 204, 29961, 97, 29956, 2708, 29948, 5, 881, 29953, 8381, 29950, 10062, 29997, 29948, 28, 260, 57, 29982, 18, 136, 29973, 29997, 1312, 989, 5046, 392, 30201, 29948, 5, 8381, 29950, 15024, 29948, 30207, 29957, 299, 10299, 27674, 29990, 14, 30833, 278, 30425, 29957, 84, 30128, 27805, 29973, 27674, 29947, 1028, 2158, 30078, 29950, 8381, 9972, 29951, 29962, 2185, 29950, 97, 29965, 913, 1086, 5, 8381, 9972, 29961, 462, 29982, 2757, 943, 4613, 3306, 9563, 11560, 2883, 90, 29982, 30277, 548, 18521, 29973, 1583, 14726, 30108, 29954

In [106]:
class Dataset_base(Dataset):
    def __init__(self, contexts, questions, answers, model_max_position_embedings, tokenizer):
        self.tokenizer = tokenizer
        self.answers = answers
        self.questions = questions
        self.contexts = contexts
        self.model_max_position_embedings = model_max_position_embedings
        print("Tokenizing ...")
        self.encodings = self.tokenizer(self.contexts, 
                                        self.questions,
                                        max_length=512, #512 truncation // 
                                        truncation=True,
                                        padding="max_length",
                                        return_token_type_ids=False)
        print("Done !!!")
        self.add_token_positions()
        
    def add_token_positions(self):
        start_positions = []
        end_positions = []
        for i in range(len(self.answers)):
            start_positions.append(self.encodings.char_to_token(i, self.answers[i]['answer_start']))
            end_positions.append(self.encodings.char_to_token(i, self.answers[i]['answer_end'] - 1)) # -1으로 : 진짜로 답이 있는 end_position 의 인덱스를 구함.(char_to_token은 인덱스를 구함)
            #https://huggingface.co/docs/tokenizers/v0.13.2/en/api/encoding#tokenizers.Encoding.char_to_token

            # positions 값이 None 값이라면, answer가 포함된 context가 잘렸다는 의미
            if start_positions[-1] is None:
                print("there is an error 1")
                start_positions[-1] = self.model_max_position_embedings
            if end_positions[-1] is None:
                print("there is an error 2")
                end_positions[-1] = self.model_max_position_embedings

        self.encodings.update({'start_positions': start_positions, 'end_positions': end_positions})

        
    def get_data(self):
        return {"contexts":self.contexts, 'questions':self.questions, 'answers':self.answers}
    
    
    def get_encodings(self):
        return self.encodings
        
    
    def __getitem__(self, idx):
        return {key:torch.tensor(val[idx]) for key, val in self.encodings.items()}
    
    def __len__(self):
        return len(self.encodings['input_ids'])

In [107]:
train_contexts, train_questions, train_answers, valid_contexts, valid_questions, valid_answers = read_input("train.json")
add_end_idx(train_answers, train_contexts)                                                      #anwer 마다 answer_end 달아준다.
train_dataset = Dataset_base(train_contexts, train_questions, train_answers, 512, tokenizer)

add_end_idx(valid_answers, valid_contexts)                                                      #anwer 마다 answer_end 달아준다.
valid_dataset = Dataset_base(valid_contexts, valid_questions, valid_answers, 512, tokenizer)

  0%|          | 0/8811 [00:00<?, ?it/s]

  0%|          | 0/978 [00:00<?, ?it/s]

Tokenizing ...
Done !!!
there is an error 2
Tokenizing ...
Done !!!


In [108]:
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

In [ ]:
!pip install wandb -qqq
import wandb
wandb.login()

^C


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: jeonht98 (goorm_project). Use `wandb login --relogin` to force relogin


True

In [ ]:
sweep_config = {
      'name' : 'project2_TEST1.ipynb',
      'method' : 'grid',
      'metric':{
          'name': 'total_valid_loss',
          'goal': 'minimize'  
      },
      'parameters' : {
          'learning_rate' : {
              'values' : [1e-4, 2.5e-5, 5e-5, 7.5e-5,1e-5]  
          },   
          'batch_size' :{
              'values' : [4,8,16]
          },
          'epochs' : {
              'values' : [2] 
          }
      }
}

sweep_id = wandb.sweep(sweep_config)

Create sweep with ID: av91nyj1
Sweep URL: https://wandb.ai/goorm_project/uncategorized/sweeps/av91nyj1


In [109]:
EPOCH = 1
LEARNING_RATE = 5e-5
BATCH_SIZE = 32


In [110]:
# Levenshtein_distance (Evaluation)

import numpy 
import torch
import os

def levenshtein_distance(s1,s2, debug=False): #레벤슈타인 거리 eval / 정답 s1과 도출한 모델 s2 비교 평가
    if len(s1) < len(s2):
        return levenshtein_distance(s2, s1, debug)

    if len(s2) == 0:
        return len(s1)

    previous_row = range(len(s2) + 1)
    for i, c1 in enumerate(s1):
        current_row = [i + 1]
        for j, c2 in enumerate(s2):
            insertions = previous_row[j + 1] + 1
            deletions = current_row[j] + 1
            substitutions = previous_row[j] + (c1 != c2)
            current_row.append(min(insertions, deletions, substitutions))

        if debug:
            print(current_row[1:])

        previous_row = current_row

    return previous_row[-1] # levenshtein_distance 값 출력력


def LD_SCORE(start_positions, end_positions, input_ids, STA_logits, END_logits):

    answer1 = [] # 정답 저장
    answer2 = [] # 예측 정답 저장

    if len(input_ids) != BATCH_SIZE: #오류 해결
       print("input_ids ERROR")
       return 0

    for i in range(BATCH_SIZE): # 기존 정답 // index 1 is out of bounds for dimension 0 with size 1 오류 발생
        if input_ids[i] == [] :
            print("input_ids ERROR")
            continue       
        else:
            PRED_IDE = input_ids[i][start_positions[i]: end_positions[i] + 1]
            PRED_ANS = tokenizer.decode(PRED_IDE)
            answer1.append(PRED_ANS)
            #print(PRED_ANS)

    for i in range(BATCH_SIZE): # 예측 정답
        if input_ids[i] == [] :
            print("input_ids ERROR")
            continue
        else:
            TK_start_index, TK_end_index = STA_logits.argmax(dim=-1), END_logits.argmax(dim=-1)
            PRED_IDE2 = input_ids[i][TK_start_index[i]: TK_end_index[i] + 1]
            PRED_ANS2 = tokenizer.decode(PRED_IDE2)
            answer2.append(PRED_ANS2)
            # print(PRED_ANS2)

    batch_score = LD_comparison(answer1, answer2)

    return batch_score


def LD_comparison(answer1,answer2):

    # 배치마다 레벤슈타인 거리 평균 구해서 출력. (train, valid 과정에서 배치마다 평균거리 구하고.)
    # train 전체 평균 거리
    # valid 전체 평균 거리

    batch_LD_score = []

    for i in range(BATCH_SIZE):
        if answer1[i] == answer2[i]: # 같으면 LD 구하는 과정 생략.
            batch_LD_score.append(0)
        else:
            batch_LD_score.append(levenshtein_distance(answer1[i],answer2[i]))

    sum_LD_score = sum(batch_LD_score)
    LD_avg = sum_LD_score / BATCH_SIZE # 배치의 레벤슈타인 거리 평균
    #print(LD_avg)

    return LD_avg



'''
levenshtein_distance 값이 튀는 현상 방지.
 - 길이가 너무 긴 정답 삭제 max_length = 20
 - 정답과 어느정도 길이차이나는 값 삭제(위와 동일) 5.9 -> 2정도 차이나는 값 삭제 7.9
 

자연어처리
자연어처리과정 2
0 5

정답 길이보다 예측 정답 길이가 길지만 2배이상 차이가 나지 않는 경우에
0으로 처리하면 LD_SCORE가 더 안나옴
따라서 위 상황에서는 1.5 ~ 1.8 정도 해당하는 길이로 잘라서 반환.

test 답 X
train 과정 나온 답 -> 바꿔서 바꾼 데이터로 


** max_len = 5.9 * 2 = 12

train 2번
1 train 원래 데이터.
2 train LD 변환한 데이터로 한번.

'''



'\nlevenshtein_distance 값이 튀는 현상 방지.\n - 길이가 너무 긴 정답 삭제 max_length = 20\n - 정답과 어느정도 길이차이나는 값 삭제(위와 동일) 5.9 -> 2정도 차이나는 값 삭제 7.9\n \n\n자연어처리\n자연어처리과정 2\n0 5\n\n정답 길이보다 예측 정답 길이가 길지만 2배이상 차이가 나지 않는 경우에\n0으로 처리하면 LD_SCORE가 더 안나옴\n따라서 위 상황에서는 1.5 ~ 1.8 정도 해당하는 길이로 잘라서 반환.\n\ntest 답 X\ntrain 과정 나온 답 -> 바꿔서 바꾼 데이터로 \n\n\n** max_len = 5.9 * 2 = 12\n\ntrain 2번\n1 train 원래 데이터.\n2 train LD 변환한 데이터로 한번.\n\n'

In [85]:
# LD SCORE 저장
train_LD_avg = [] 
valid_LD_avg = []

def train_runner(model, train_dataset, valid_dataset , batch_size, num_train_epochs, learning_rate):

    #wandb.init(project = 'project2_test1',reinit=True)

    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    
    model.to(device)
    model.train()
    train_dataloader = DataLoader(dataset=train_dataset, batch_size=batch_size)
    valid_dataloader = DataLoader(dataset = valid_dataset, batch_size = batch_size)

    lowest_total_valid_loss = 9999.

    global_total_step = len(train_dataloader) * num_train_epochs
    optimizer = AdamW(model.parameters(), lr=learning_rate, weight_decay=0)
    print("TRAIN START")
    with tqdm(total=global_total_step, unit='step') as t:
        total = 0
        total_loss = 0
        for epoch in range(num_train_epochs):
            for iteration,batch in enumerate(train_dataloader):
                optimizer.zero_grad()
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                start_positions = batch['start_positions'].to(device)
                end_positions = batch['end_positions'].to(device)                

                outputs = model(input_ids,
                             attention_mask=attention_mask,
                             start_positions=start_positions,
                             end_positions=end_positions)
                
                ### LD_SCORE - train
                STA_logits, END_logits = outputs.start_logits, outputs.end_logits
                score_save1 = LD_SCORE(start_positions, end_positions, input_ids, STA_logits, END_logits)
                train_LD_avg.append(score_save1)
                #wandb.log({'train_batch_LD':score_save1})

                ####
                #loss_logit_change(start_positions, end_positions, input_ids, STA_logits, END_logits)
                loss = outputs.loss
                #wandb loss
                #wandb.log({'Train_loss':loss.item()})

                loss.backward()
                optimizer.step()
                
                batch_loss = loss.item() * len(input_ids)
                total += len(input_ids)
                total_loss += batch_loss
                global_total_step += 1
                t.set_postfix(loss="{:.6f}".format(total_loss / total), batch_loss="{:.6f}".format(batch_loss))
                t.update(1)

                # wandb loss
                #wandb.log({'Train_batch_loss':batch_loss})
                #wandb.log({'LOSS':total_loss / total})
                
                del input_ids
                del attention_mask
                del start_positions
                del end_positions
                del outputs
                del loss

                ## validation ##
                if iteration != 0 and iteration % int(len(train_dataloader) / 5) == 0:
                    total_valid_loss = 0
                    for batch_val in valid_dataloader:
                        model.eval()
                        optimizer.zero_grad()

                        input_ids = batch_val['input_ids'].to(device)
                        attention_mask = batch_val['attention_mask'].to(device)
                        start_positions = batch_val['start_positions'].to(device)
                        end_positions = batch_val['end_positions'].to(device)
                
                        with torch.no_grad():
                            outputs = model(input_ids,
                                    attention_mask=attention_mask,
                                    start_positions=start_positions,
                                    end_positions=end_positions)
                            
                            ### LD_SCORE - valid
                            STA_logits, END_logits = outputs.start_logits, outputs.end_logits
                            score_save2 = LD_SCORE(start_positions, end_positions, input_ids, STA_logits, END_logits)
                            valid_LD_avg.append(score_save2)

                            # wandb LD
                            #wandb.log({'Valid_batch_LD':score_save2})

                            loss = outputs.loss
                            total_valid_loss += loss.item()

                            # wandb loss
                            #wandb.log({'Valid_loss':loss.item()})
                            #wandb.log({'total_valid_loss':total_valid_loss})
                    
                    if total_valid_loss < lowest_total_valid_loss:
                        print(f"lowest_total_valid_loss: {total_valid_loss} epoch : {epoch} iteration : {iteration}")
                        torch.save(model.state_dict(),'./output_model_best')
                        lowest_total_valid_loss = total_valid_loss
                ## validation ##

    #model.save_pretrained("./klue_output_model")
    # train, valid 평균 LD 거리
    train_AVG = sum(train_LD_avg) / len(train_LD_avg)
    valid_AVG = sum(valid_LD_avg) / len(valid_LD_avg)
    #wandb.log({'train_AVG_LD':train_AVG})
    #wandb.log({'valid_AVG_LD':valid_AVG})

    print('train LD average',train_AVG,'valid LD average',valid_AVG)
    print("TRAIN END")

In [ ]:
# wandb sweep train 실행.

# wandb.agent( sweep_id , function=train_runner, count=1)

In [111]:
train_runner(model,train_dataset,valid_dataset, BATCH_SIZE, EPOCH, LEARNING_RATE)

TRAIN START


  0%|          | 0/497 [00:00<?, ?step/s]

input_ids ERROR
lowest_total_valid_loss: 127.19607412815094 epoch : 0 iteration : 99
input_ids ERROR
lowest_total_valid_loss: 113.00674819946289 epoch : 0 iteration : 198
input_ids ERROR
lowest_total_valid_loss: 106.80285608768463 epoch : 0 iteration : 297
input_ids ERROR
lowest_total_valid_loss: 102.56842559576035 epoch : 0 iteration : 396
input_ids ERROR
lowest_total_valid_loss: 101.88247060775757 epoch : 0 iteration : 495
input_ids ERROR
train LD average 14.711274509803921 valid LD average 11.375145687645688
TRAIN END


In [112]:
MAX_LEN = 10

# train / valid 과정에서 나온 이상값 제거 (정확한 점수 X)

train_LD_avg = [v for v in train_LD_avg if v < MAX_LEN]  
TR_AVG = sum(train_LD_avg) / len(train_LD_avg) 

valid_LD_avg = [v for v in valid_LD_avg if v < MAX_LEN]
VA_AVG = sum(valid_LD_avg) / len(valid_LD_avg)

print('predicted Train Avgrage LD',TR_AVG,'predicted Valid Avgrage LD',VA_AVG) # 예측값 확인.

predicted Train Avgrage LD 5.467939121756487 predicted Valid Avgrage LD 5.530490451388889


In [113]:
def read_dev(path):
    with open(path, 'rb') as f:
        kdict = json.load(f)

    contexts = []
    questions = []
    answers = []
    guids = []

    for group in tqdm(kdict['data']):
        for passage in group['paragraphs']:
            context = passage['context']
            for qa in passage['qas']:
                question = qa['question']
                guid = qa['guid']
                #temp_answer = []
                #for answer in qa['answers']:
                    #temp_answer.append(answer['text'])
                #if len(temp_answer) != 0: # answers의 길이가 0 == 답변할 수 없는 질문
                    #contexts.append(context)
                    #questions.append(question)
                    #answers.append(temp_answer)
                contexts.append(context)##
                questions.append(question)##
                guids.append(guid)

    #return contexts, questions, answers
    return contexts, questions , guids

In [114]:
#dev_contexts, dev_questions, dev_answers = read_dev("test.json")
dev_contexts, dev_questions, dev_guids = read_dev("test.json")

  0%|          | 0/3709 [00:00<?, ?it/s]

In [115]:
import re
def remove_post(text):
        ''' 불필요한 기호 제거 '''
        text = text.strip()
        text = re.sub("'", "", text)
        text = re.sub('"', "", text)
        text = re.sub('《', "", text)
        text = re.sub('》', "", text)
        text = re.sub('<', "", text)
        text = re.sub('>', "", text)
        text = re.sub('〈', "", text)
        text = re.sub('〉', "", text)
        text = re.sub("\(", "", text)
        text = re.sub("\)", "", text)
        text = re.sub("‘", "", text)
        text = re.sub("’", "", text)
        text = re.sub("  ", " ", text)
        text = re.sub("#", "", text)
        return text

In [116]:
'''
idea  
틀린 답을 바로 빈칸으로 내보내는게 과연 이득일까?
logit 확률값을 적용해서 틀린답이면 다음으로 높은 확률값이 정답일수도 있지않을까?
top 5의 확률값을 다 적용해도 틀린 답이라면 그때 빈칸을 내보내도 괜찮을듯.
'''
# start, end logit의 확률값을 이용한 예측 정답값
# logit의 상위 5개 확률을 리스트로 뽑아 틀린 정답이었다면 다음 확률로 넘어가서 확인.

def to_list(tensor):
    return tensor.detach().cpu().tolist()

def logits_change(input_ids, STA_logits, END_logits):

    # 로짓의 확률값 ~ 상위 5개를 선택 
    # 틀린 추론이었다면 다음 선택 (틀린 추론 : start > end, 길이가 너무 긴 문장.)
    change_logit = 0
    cnt = 0
    
    # 기존 정답
    save_s = STA_logits
    save_e = END_logits

    STK_start_index, STK_end_index = save_s.argmax(dim=-1), save_e.argmax(dim=-1)
    save_pred_ids = tokenizer.decode(input_ids[0][STK_start_index: STK_end_index + 1])
    #print(save_pred_ids) 

    # 바뀐 정답
    # Tensor형태의 logit의 확률값을 리스트로 만들어줌 
    STA_logits = to_list(STA_logits)[0]
    END_logits = to_list(END_logits)[0]

    # 리스트로 만든 확률값 -> 큰 순서대로 정렬 + 인덱스 
    start_idx_and_logit = sorted(enumerate(STA_logits), key=lambda x: x[1], reverse=True)
    end_idx_and_logit = sorted(enumerate(END_logits), key=lambda x: x[1], reverse=True)

    # 확률값 큰 순서대로 Top 5 
    start_idx_and_logit = start_idx_and_logit[:5]
    end_idx_and_logit = end_idx_and_logit[:5]

    TK_start_index, TK_end_index = start_idx_and_logit, end_idx_and_logit

    # 확률이 높은 순서대로 점검. 틀린 값이면 다음 확률로 넘어가고 맞는 답이면 저장후 반복문 종료
    for i in range(5):
        if TK_start_index[i][0] > TK_end_index[i][0] or TK_end_index[i][0] - TK_start_index[i][0] > 10 : 
            cnt += 1
            continue
        else : 
            change_logit += 1
            pred_ids = input_ids[0][TK_start_index[i][0]: TK_end_index[i][0] + 1]
            pred_ids = tokenizer.decode(pred_ids)
            #print(pred_ids, 'change')
            break

    if change_logit == 0 :
        return save_pred_ids
    elif cnt == 5:
        return ''
    else : 
        if pred_ids == save_pred_ids:
            #print('same answer')
            return pred_ids
        else :
            #print('different answer')
            return pred_ids

    

# start_logits , end_logits
# index를 추적하면서 시작, 종료 index에 대한 확률이 가장 높은것을 선택하는 방법.
# 만약에 차이가 큰 start, end 값을 반환할때 이 정보들을 저장하지 않고 넘긴다면? 


In [117]:
def prediction(contexts, questions, guids):
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

    model.load_state_dict(torch.load('./output_model_best'))
    model.to(device)
    
    model.eval()
    
    result = []
    
    with torch.no_grad():
        
        for context, question, guid in zip(contexts, questions, guids):
            encodings = tokenizer(context, question, max_length=512, truncation=True,
                                     padding="max_length", return_token_type_ids=False)
            encodings = {key: torch.tensor([val]) for key, val in encodings.items()}
            
            input_ids = encodings["input_ids"].to(device)
            attention_mask = encodings["attention_mask"].to(device)
            
            outputs = model(input_ids, attention_mask=attention_mask)
            start_logits, end_logits = outputs.start_logits, outputs.end_logits

            ######
            pred =logits_change(input_ids, start_logits, end_logits)
            '''
            token_start_index, token_end_index = start_logits.argmax(dim=-1), end_logits.argmax(dim=-1)
            pred_ids = input_ids[0][token_start_index: token_end_index + 1]
            
            if token_start_index > token_end_index:
                pred = ''
            else:
                pred = tokenizer.decode(pred_ids)
                pred = pred[:10] # 1.5~1.8
            '''
            


            pred = pred[:8] # 6~12 테스트 결과 8이 가장 좋은 점수.
            pred = remove_post(pred)

            tp = (guid,pred)
            
            result.append(tp)
    return result

In [118]:
pred_answers = prediction(dev_contexts, dev_questions, dev_guids)

In [119]:
import csv
f = open('sangrimlee_bert-base-multilingual-cased-korquad_validation_pred_truncation.csv','w', newline='')
wr = csv.writer(f)
wr.writerow(['Id','Predicted'])

for tp in pred_answers:
    wr.writerow([tp[0],tp[1]])

f.close()

In [120]:
# pred_answers에서 도출한 정답 비교
import csv

def calculate_Leven(source, ref, result_file):
    with open(source, 'r') as input1:
        with open(ref, 'r') as input2:
            with open(result_file, 'w') as csvoutput:
                reader1 = csv.reader(input1)
                reader2 = list(csv.reader(input2))
                writer = csv.writer(csvoutput)
                result = []
                mean = []
                headers = next(reader1)
                result.append(headers)
                index = 0
                for row1 in reader1:
                    #print("First row")
                    #print(row1[1])
                    index+=1
                    #print(reader2[index][1])
                    a = levenshtein_distance(row1[1], reader2[index][1])
                    #print(row1[1],'////',reader2[index][1])
                    '''
                    max = 0
                    while max < 1:
                        for row2 in reader2:
                            a = distance(row1[1],row2[1])
                            print(a)
                            b = 1 - a/len(row1[1])
                            if b > max:
                                max = b
                                SKU = row2[1]
                    '''
                    mean.append(a)
                    row1.append(a)
                    result.append(row1)
                mean1 = sum(mean) / len(mean)
                print(mean1)
                writer.writerows(result)


In [121]:
calculate_Leven('klue.csv', 'sangrimlee_bert-base-multilingual-cased-korquad_validation_pred_truncation.csv', 'result_file.csv')

3.1137724550898205


In [ ]:
# logit 값에 대한 soft voting

def to_list(tensor):
    return tensor.detach().cpu().tolist()

def logits_voting(A_model, B_model, C_model):
    # 각 모델마다 input_ids, start_logits, end_logits값 저장.
    # 모델 logit 확률 top 5 구해서 모델 1 top1 모델 2 top1 모델3 top1 모델1 top2 순으로 체크.

    